In [1]:
import os
os.chdir("/home/maikfangogoair/tensorflow/tensorflow/examples/speech_commands")

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os.path
import sys

import numpy as np
from six.moves import xrange  # pylint: disable=redefined-builtin
import tensorflow as tf

import input_data
import models
from tensorflow.python.platform import gfile

In [16]:
wanted_words="yes,no,up,down,left,right,on,off,stop,go"
sample_rate=16000#Expected sample rate of the wavs
clip_duration_ms=1000#Expected duration in milliseconds of the wavs
window_size_ms=30.0#How long each spectrogram timeslice is
window_stride_ms=10.0#How long each spectrogram timeslice is
dct_coefficient_count=40#How many bins to use for the MFCC fingerprint
data_url="http://download.tensorflow.org/data/speech_commands_v0.01.tar.gz"
data_dir="/home/maikfangogoair/tmp"
silence_percentage=10.0 #How much of the training data should be silence.
unknown_percentage=10.0 #How much of the training data should be unknown words.
validation_percentage=10 #What percentage of wavs to use as a validation set.
testing_percentage=10 #What percentage of wavs to use as a test set.
time_shift_ms=100.0 #Range to randomly shift the training audio by in time.
batch_size=100
background_frequency=0.8#How many of the training samples have background noise mixed in.
background_volume=0.1#How loud the background noise should be, between 0 and 1

In [5]:
model_settings = models.prepare_model_settings(
    len(input_data.prepare_words_list(wanted_words.split(','))),
    sample_rate, clip_duration_ms, window_size_ms,
    window_stride_ms, dct_coefficient_count)
audio_processor = input_data.AudioProcessor(
    data_url, data_dir, silence_percentage,
    unknown_percentage,
    wanted_words.split(','), validation_percentage,
    testing_percentage, model_settings)
fingerprint_size = model_settings['fingerprint_size']
label_count = model_settings['label_count']
time_shift_samples = int((time_shift_ms * sample_rate) / 1000)

40 98


In [14]:
print(fingerprint_size)
print(label_count)
print(time_shift_samples)
print(model_settings)
print(audio_processor.set_size("training"))
print(audio_processor.set_size("validation"))
print(audio_processor.set_size("testing"))

3920
12
1600
{'fingerprint_size': 3920, 'spectrogram_length': 98, 'dct_coefficient_count': 40, 'window_size_samples': 480, 'window_stride_samples': 160, 'sample_rate': 16000, 'label_count': 12, 'desired_samples': 16000}
22246
3093
3081


In [15]:
sess = tf.InteractiveSession()

In [22]:
train_fingerprints, train_ground_truth = audio_processor.get_data(
    audio_processor.set_size("training"), 0, model_settings, background_frequency,
    background_volume, time_shift_samples, 'training', sess)
validation_fingerprints, validation_ground_truth = (
            audio_processor.get_data(audio_processor.set_size("validation"), 0, model_settings, 0.0,
                                     0.0, 0, 'validation', sess))
test_fingerprints, test_ground_truth = audio_processor.get_data(
    audio_processor.set_size("testing"), 0, model_settings, 0.0, 0.0, 0, 'testing', sess)

In [31]:
print(train_fingerprints.shape[0])
print(validation_fingerprints.shape[0])
print(test_fingerprints.shape[0])

22246
3093
3081


In [26]:
dir="/home/maikfangogoair/tmp/"
np.save(dir+"train_fingerprints.npy",train_fingerprints)
np.save(dir+"train_ground_truth.npy",train_ground_truth)
np.save(dir+"validation_fingerprints.npy",validation_fingerprints)
np.save(dir+"validation_ground_truth.npy",validation_ground_truth)
np.save(dir+"test_fingerprints.npy",test_fingerprints)
np.save(dir+"test_ground_truth.npy",test_ground_truth)

In [41]:
print(audio_processor.words_list)
print(audio_processor.word_to_index)
print(len(audio_processor.background_data))

['_silence_', '_unknown_', 'yes', 'no', 'up', 'down', 'left', 'right', 'on', 'off', 'stop', 'go']
{'zero': 1, 'nine': 1, 'house': 1, 'left': 6, 'two': 1, 'on': 8, 'down': 5, 'marvin': 1, 'up': 4, 'three': 1, 'no': 3, 'happy': 1, 'bed': 1, '_silence_': 0, 'tree': 1, 'wow': 1, 'cat': 1, 'sheila': 1, 'seven': 1, 'five': 1, 'four': 1, 'one': 1, 'six': 1, 'dog': 1, 'go': 11, 'yes': 2, 'stop': 10, 'right': 7, 'eight': 1, 'bird': 1, 'off': 9}
6


In [62]:
print(len(audio_processor.data_index['training']))
print(train_ground_truth.shape)

22246
(22246,)


In [59]:
print([x['label']for x in audio_processor.data_index['training'][:10]])
print(train_ground_truth[:10])

['bird', 'no', 'no', 'go', 'no', '_silence_', 'down', 'happy', 'right', 'no']
[  9.   1.   8.   7.   3.   2.  11.   4.   3.   3.]
